In [1]:
import numpy as np
from copy import deepcopy
import itertools
import matplotlib
from numpy import radians as rad
from matplotlib.animation import FuncAnimation
from scipy.ndimage import convolve,convolve1d
import cmasher
import seaborn as sns
from collections import defaultdict
from manim import *
import networkx as nx
from scipy.interpolate import interp1d
import itertools
import networkx as nx
import matplotlib.pyplot as plt
import random
from networkx.drawing.nx_pydot import graphviz_layout
from scipy.special import softmax
import itertools as it
from collections import deque
import torch.nn.functional as F
import pandas as pd
from scipy.integrate import quad

In [75]:
import itertools

# Derangements

In [118]:
# A) Pakkeleg - introducing the problem. First the characters and the presents, they take the presents. 
# They shuffled a few times, then the derangement and the problem.
# New plan: just do the basic inclusion-exclusion. SO BASIC

In [84]:
class Intro(Scene):
    
    def construct(self):
        # Create viking images
        viking_yellow = ImageMobject("images/yellow_viking.png").scale(0.5)
        viking_blue = ImageMobject("images/blue_viking.png").scale(0.5)
        viking_red = ImageMobject("images/red_viking.png").scale(0.5)
        viking_green = ImageMobject("images/green_viking.png").scale(0.5)

        viking_yellow.shift(LEFT * 3)
        viking_blue.next_to(viking_yellow, RIGHT, buff=1)
        viking_red.next_to(viking_blue, RIGHT, buff=1)
        viking_green.next_to(viking_red, RIGHT, buff=1)
        
        vikings = Group(viking_yellow, viking_blue, viking_red, viking_green).move_to(UP)

        present_yellow = ImageMobject("images/yellow_present.png").scale(0.3)
        present_blue = ImageMobject("images/blue_present.png").scale(0.3)
        present_red = ImageMobject("images/red_present.png").scale(0.3)
        present_green = ImageMobject("images/green_present.png").scale(0.3)

        direction = DOWN
        present_yellow.next_to(viking_yellow, direction)
        present_blue.next_to(viking_blue, direction)
        present_red.next_to(viking_red, direction)
        present_green.next_to(viking_green, direction)
        
        presents = Group(present_yellow, present_blue, present_red, present_green)

        # Animations
        self.play(FadeIn(vikings))  # Show vikings together
        self.play(FadeIn(presents))  # Show presents together, positioned next to vikings

        # Pause to display final arrangement
        self.wait(1)
        # Animate presents flying to the bottom to form a pile
        pile_position = DOWN * 3
        self.play(
            present_yellow.animate.move_to(pile_position + LEFT * 0.5),
            present_blue.animate.move_to(pile_position + RIGHT * 0.5),
            present_red.animate.move_to(pile_position + LEFT * 1.5),
            present_green.animate.move_to(pile_position + RIGHT * 1.5),
        )

        # Pause to display the pile
        self.wait(1)

        dices = [
            ImageMobject("images/dice_one.png").scale(0.5),
            ImageMobject("images/dice_two.png").scale(0.5),
            ImageMobject("images/dice_three.png").scale(0.5),
            ImageMobject("images/dice_four.png").scale(0.5),
            ImageMobject("images/dice_five.png").scale(0.5),
            ImageMobject("images/dice_six.png").scale(0.5),
        ]

        self.add(dices[0].next_to(viking_yellow, UP))
        self.wait(1)
        self.remove(dices[0])
        self.wait(0.5)

        self.add(dices[4].next_to(viking_blue, UP))
        self.wait(1)
        self.remove(dices[4])
        self.wait(0.5)

        self.add(dices[5].next_to(viking_red, UP))
        self.wait(1)
        self.play(present_yellow.animate.next_to(viking_red, DOWN))
        self.wait(0.5)
        
        self.remove(dices[5])
        self.wait(0.5)

        self.add(dices[1].next_to(viking_green, UP))
        self.wait(1)
        self.remove(dices[1])
        self.wait(0.5)

        self.add(dices[5].next_to(viking_yellow, UP))
        self.wait(1)
        self.play(present_green.animate.next_to(viking_yellow, DOWN))
        self.wait(0.5)
        
        self.remove(dices[5])
        self.wait(0.5)

        self.add(dices[3].next_to(viking_blue, UP))
        self.wait(1)
        self.remove(dices[3])
        self.wait(0.5)

        self.add(dices[5].next_to(viking_green, UP))
        self.wait(1)
        self.play(present_red.animate.next_to(viking_green, DOWN))
        self.wait(0.5)
        
        self.remove(dices[5])
        self.wait(1)

        self.play(present_blue.animate.next_to(viking_blue, DOWN))
        self.wait(1)

        blue_group = Group(present_blue, viking_blue)

        surbox = SurroundingRectangle(blue_group, color=YELLOW)
        self.play(FadeIn(surbox))

        self.wait(1)

        self.play(FadeOut(surbox))
        self.play(
            present_yellow.animate.move_to(pile_position + LEFT * 0.5),
            present_blue.animate.move_to(pile_position + RIGHT * 0.5),
            present_red.animate.move_to(pile_position + LEFT * 1.5),
            present_green.animate.move_to(pile_position + RIGHT * 1.5)
        )
        self.wait(1)

        self.play(
            present_blue.animate.next_to(viking_red, DOWN),
            present_red.animate.next_to(viking_yellow, DOWN),
            present_yellow.animate.next_to(viking_green, DOWN),
            present_green.animate.next_to(viking_blue, DOWN),
        )
        self.wait(1)

        title = Text('"derangement"', font_size=60).next_to(presents, DOWN)
        self.play(Write(title))
        self.wait(1)

        self.play(Unwrite(title), FadeOut(vikings), FadeOut(presents))
        self.wait(1)

In [85]:
%manim Intro

Manim Community v0.18.1

[01/16/25 22:42:35] INFO     Animation 0 : Using cached data (hash :                           ]8;id=225705;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=92599;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/renderer/cairo_renderer.py#88\88]8;;\
                             3977891868_285927764_223132457)                                                       

                    INFO     Animation 1 : Using cached data (hash :                           ]8;id=95832;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=815558;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/renderer/cairo_renderer.py#88\88]8;;\
                             2852726489_1279957977_3867173521)                                                     

                    INFO     Animation 2 : Using cached data (hash :                           ]8;id=943871;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=61090;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/renderer/cairo_renderer.py#88\88]8;;\
                             2852726489_2919340033_454327810)                                                      

                    INFO     Animation 3 : Using cached data (hash :                           ]8;id=234962;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=524295;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/renderer/cairo_renderer.py#88\88]8;;\
                             2852726489_3025981134_2796728358)                                                     

...

[01/16/25 22:42:44] INFO     Animation 37 : Partial movie file written in                  ]8;id=485117;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=741513;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/Users/nmb127/Documents/animations/notebooks/media/videos/no                         
                             tebooks/1080p60/partial_movie_files/Intro/2852726489_29193400                         
                             33_1755142888.mp4'                                                                    

                    INFO     Combining to Movie file.                                      ]8;id=860167;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=631814;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/scene/scene_file_writer.py#617\617]8;;\

                    INFO                                                                   ]8;id=400476;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=668776;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/scene/scene_file_writer.py#737\737]8;;\
                             File ready at                                                                         
                             '/Users/nmb127/Documents/animations/notebooks/media/videos/no                         
                             tebooks/1080p60/Intro.mp4'                                                            
                                                                                                                   

                    INFO     The partial movie directory is full (> 100 files). Therefore, ]8;id=679532;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=362446;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/scene/scene_file_writer.py#707\707]8;;\
                             manim has removed the 2 oldest file(s). You can change this                           
                             behaviour by changing max_files_cached in config.                                     

                    INFO     Rendered Intro                                                            ]8;id=110289;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=379111;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/scene/scene.py#247\247]8;;\
                             Played 38 animations                                                                  

# Permutations

In [153]:
from manim import *
import itertools

class Permutations(Scene):

    def construct(self):
        colors = ["yellow", "blue", "red", "green"]
                
        # Create present templates
        present_templates = [
            ImageMobject(f"images/{color}_present.png").scale(0.1)
            for color in colors
        ]

        spacing = 1.5  # Spacing between squares

        # Create and position squares
        squares = [
            Square(side_length=1, color=color, fill_opacity=1).shift(LEFT * (1.5 + spacing * i))
            for i, color in enumerate(reversed(colors))
        ]

        # Group squares and position them in the middle
        squares_group = Group(*squares).move_to(ORIGIN)

        # Animate squares appearing
        self.play(*[Create(square) for square in squares])
        self.wait(1)
        
        # Animate resizing and moving to the top
        self.play(
            squares_group.animate.scale(0.5),
            run_time=2
        )
        self.wait(1)
        
        # Generate all permutations
        objects = [0, 1, 2, 3]
        permutations = list(itertools.permutations(objects))

        # Grid layout parameters
        max_rows = 6  # Number of rows for the grid
        spacing_x = 3.5  # Horizontal spacing between permutations
        spacing_y = 0.5  # Vertical spacing between rows

        # Calculate offsets for each permutation
        start_x = -6  # Starting x position (left side of screen)
        start_y = 3.5   # Starting y position (top of screen)

        # Display all permutations and add counters
        all_groups = []
        positions = []
        counters = {}
        counter_texts = {}
        
        for perm_idx, perm in enumerate(permutations):
            row = perm_idx % max_rows
            col = perm_idx // max_rows

            x_offset = start_x + col * spacing_x
            y_offset = start_y - row * spacing_y

            presents = [
                present_templates[idx].copy().move_to([x_offset + i * 0.5, y_offset, 0])
                for i, idx in enumerate(perm)
            ]

            group = Group(*presents)
            all_groups.append((group, perm))
            positions.append(group.get_center())
            counters[perm] = 0

            # Add counter next to the permutation
            counter_text = Text(f"{counters[perm]}", font_size=24).next_to(group, RIGHT).shift(RIGHT * 0.5)
            counter_texts[perm] = counter_text
            self.add(counter_text)

            self.play(FadeIn(group), run_time=0.05)

        self.wait(1)

        # Animate moving permutations with `1` at index 1
        for index in range(4):  # Simulate multiple rounds of moving
            bottom_y_offset = -1
            moved_groups = []
            moved_positions = []
            for group, perm in all_groups:
                if perm[index] == index:
                    moved_positions.append(group.get_center())
                    self.play(group.animate.move_to([0, bottom_y_offset, 0]), run_time=0.5)
                    bottom_y_offset -= 0.5  # Adjust for stacking
                    moved_groups.append(group)
                    
                    # Update counter without moving its position
                    counters[perm] += 1
                    counter_texts[perm].become(
                        Text(f"{counters[perm]}", font_size=24).move_to(counter_texts[perm].get_center())
                    )
            
            self.wait(2)

            # Animate moving them back to their original positions
            for group, position in zip(moved_groups, moved_positions):
                self.play(group.animate.move_to(position), run_time=0.5)

            self.wait(2)


In [154]:
%manim Permutations

Manim Community v0.18.1

[01/18/25 18:40:15] INFO     Animation 0 : Using cached data (hash :                           ]8;id=186233;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=459332;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/renderer/cairo_renderer.py#88\88]8;;\
                             3977891868_2120542793_223132457)                                                      

                    INFO     Animation 1 : Using cached data (hash :                           ]8;id=221899;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=453122;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/renderer/cairo_renderer.py#88\88]8;;\
                             2852726489_2919340033_3650284402)                                                     

                    INFO     Animation 2 : Using cached data (hash :                           ]8;id=764104;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=794965;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/renderer/cairo_renderer.py#88\88]8;;\
                             2852726489_202575717_4227516557)                                                      

                    INFO     Animation 3 : Using cached data (hash :                           ]8;id=823333;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=730373;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/renderer/cairo_renderer.py#88\88]8;;\
                             2852726489_2919340033_151432698)                                                      

...

[01/18/25 18:47:07] INFO     Animation 70 : Partial movie file written in                  ]8;id=828199;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=133300;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/Users/nmb127/Documents/animations/notebooks/media/videos/no                         
                             tebooks/1080p60/partial_movie_files/Permutations/2852726489_7                         
                             84179552_1356048833.mp4'                                                              

                    INFO     Combining to Movie file.                                      ]8;id=548146;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=611058;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/scene/scene_file_writer.py#617\617]8;;\

[01/18/25 18:47:08] INFO                                                                   ]8;id=958879;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=855379;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/scene/scene_file_writer.py#737\737]8;;\
                             File ready at                                                                         
                             '/Users/nmb127/Documents/animations/notebooks/media/videos/no                         
                             tebooks/1080p60/Permutations.mp4'                                                     
                                                                                                                   

                    INFO     The partial movie directory is full (> 100 files). Therefore, ]8;id=152795;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=752629;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/scene/scene_file_writer.py#707\707]8;;\
                             manim has removed the 67 oldest file(s). You can change this                          
                             behaviour by changing max_files_cached in config.                                     

                    INFO     Rendered Permutations                                                     ]8;id=442662;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=290527;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/scene/scene.py#247\247]8;;\
                             Played 71 animations                                                                  